# Basic Q Learner
- Tutorial from deeplizar.com
- https://deeplizard.com/learn/video/HGeI30uATws

### Import Libraries

In [1]:
import numpy as np
import gym
import random
import time
from IPython.display import clear_output

### Creating the environment

In [2]:
env = gym.make("FrozenLake-v0")

### Creating the Q-Table
A Q-Table is matrix that maps every state with every action 

In [3]:
# Get the total actions that we can do 
action_space_size = env.action_space.n
# Get all the possible states
state_space_size = env.observation_space.n
# Create the table
q_table = np.zeros((state_space_size,action_space_size))

### Printing the Q-Table

In [4]:
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


### Initializing Q-Learning Parameters
- num_episodes = number of episodes we want the agent to play during training
- max_steps_per_episode = we define a maximum number of steps that our agent is allowed to take within a single episode
- learning_rate = alpha symbol from Bellman Equation
- discount_rate = gamma symbol from Bellma Equation

#### The next four parameretes are all related to exploration-exploitation
- The max and min are just bounds to how large or small our exploration rate can be. Remember, the exploration rate was represented with the symbol E(in greek) when we discussed it previously.
- exploration_decay_rate to 0.01 to determine the rate at which the exploration_rate will decay

\begin{equation*} q^{new}\left( s,a\right) =\left( 1-\alpha \right) ~\underset{\text{old value} }{\underbrace{q\left( s,a\right) }\rule[-0.05in]{0in}{0.2in} \rule[-0.05in]{0in}{0.2in}\rule[-0.1in]{0in}{0.3in}}+\alpha \overset{\text{ learned value}}{\overbrace{\left(
                                                R_{t+1}+\gamma \max_{a^{^{\prime }}}q\left( s^{\prime },a^{\prime }\right) \right) }} \end{equation*}

In [5]:
num_episodes = 10000
max_steps_per_episode = 100

learning_rate = 0.1
discount_rate = 0.99

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.01

### Coding The Q-Learning Algorithm Training Loop

In [6]:
# List to hold all of the rewards we’ll get from each episode.
# This will be so we can see how our game score changes over time.
rewards_all_episodes = []

In [7]:
# Q-learning algorithm
for episode in range(num_episodes):
    # initialize new episode params
    state = env.reset()
    # The done variable just keeps track of whether or not our episode is finished
    done = False
    # We need to keep track of the rewards within the current episode as well,
    # so we set rewards_current_episode to 0 since we start out with no rewards
    # at the beginning of each episode.
    rewards_current_episode = 0

    for step in range(max_steps_per_episode): 
        # Exploration-exploitation trade-off (Epsilon Greedy Strategy)
        exploration_rate_threshold = random.uniform(0, 1)
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:])# Exploit
        else:
            action = env.action_space.sample()# Explore
        # Take new action
        new_state, reward, done, info = env.step(action)
        # Update Q-table
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))
        # Set new state
        state = new_state
        # Add new reward
        rewards_current_episode += reward
        if done == True: 
            break
    # Exploration rate decay (Epsilon Greedy Strategy)
    exploration_rate = min_exploration_rate + (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)
    # Add current episode reward to total rewards list
    rewards_all_episodes.append(rewards_current_episode)
    

### After all episodes complete

In [8]:
# Calculate and print the average reward per thousand episodes
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/1000)
count = 1000

print("********Average reward per thousand episodes********\n")
for r in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000

********Average reward per thousand episodes********

1000 :  0.1300000000000001
2000 :  0.3910000000000003
3000 :  0.5540000000000004
4000 :  0.5540000000000004
5000 :  0.5430000000000004
6000 :  0.6150000000000004
7000 :  0.6260000000000004
8000 :  0.6920000000000005
9000 :  0.7150000000000005
10000 :  0.6850000000000005


### Interpreting the training results

Let’s take a second to understand how we can interpret these results. Our agent played 10,000 episodes. At each time step within an episode, the agent received a reward of 1 if it reached the frisbee, otherwise, it received a reward of 0. If the agent did indeed reach the frisbee, then the episode finished at that time-step.

So, that means for each episode, the total reward received by the agent for the entire episode is either 1 or 0. So, for the first thousand episodes, we can interpret this score as meaning that 16% of the time, the agent received a reward of 1 and won the episode. And by the last thousand episodes from a total of 10,000, the agent was winning 70% of the time.

By analyzing the grid of the game, we can see it’s a lot more likely that the agent would fall in a hole or perhaps reach the max time steps than it is to reach the frisbee, so reaching the frisbee  of the time isn’t too shabby, especially since the agent had no explicit instructions to reach the frisbee. It learned that this is the correct thing to do.

- SFFF
- FHFH
- FFFH
- HFFG

Lastly, we print out our updated Q-table to see how that has transitioned from its initial state of all zeros.

In [9]:
# Print updated Q-table
print("\n\n********Q-table********\n")
print(q_table)




********Q-table********

[[0.53582245 0.51979907 0.52721489 0.51933078]
 [0.12806382 0.26661824 0.26822145 0.50802356]
 [0.42177449 0.39301349 0.36889628 0.46498052]
 [0.27266155 0.20385012 0.31470512 0.4516567 ]
 [0.55009791 0.40111389 0.35059276 0.32504156]
 [0.         0.         0.         0.        ]
 [0.26728887 0.05444099 0.21566352 0.11153021]
 [0.         0.         0.         0.        ]
 [0.40694036 0.26034254 0.46378143 0.5632143 ]
 [0.39479447 0.58970032 0.49878607 0.44215101]
 [0.48296618 0.49704292 0.36535829 0.35428938]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.39875038 0.46915578 0.68270213 0.56277737]
 [0.75267538 0.80856986 0.75122684 0.76213033]
 [0.         0.         0.         0.        ]]


### Code to Watch the agent play the game

In [10]:
# Watch our agent play Frozen Lake by playing the best action 
# from each state according to the Q-table
for episode in range(3):
    # initialize new episode params
    state = env.reset()
    done = False
    print("*****EPISODE ", episode+1, "*****\n\n\n\n")
    time.sleep(1)

    for step in range(max_steps_per_episode):        
        # Show current state of environment on screen
        clear_output(wait=True)
        env.render()
        time.sleep(0.3)
        # Choose action with highest Q-value for current state
        action = np.argmax(q_table[state,:])  
        # Take new action
        new_state, reward, done, info = env.step(action)

        if done:
            clear_output(wait=True)
            env.render()
            if reward == 1:
                # Agent reached the goal and won episode
                print("****You reached the goal!****")
                time.sleep(3)
            else:
                # Agent stepped in a hole and lost episode 
                print("****You fell through a hole!****")
                time.sleep(3)
                clear_output(wait=True)
            break
        # Set new state
        state = new_state

env.close()

  (Down)
SFFF
FHFH
FFFH
HFFG
****You reached the goal!****
